In [1]:
import sys
sys.path.append('../')

In [11]:
from simulate import simulate_surgeries
from util import *
import seaborn as sns
import lightgbm as lgbm
import joblib

In [3]:
# # simulation
# df = simulate_surgeries(70000)
# continuous_columns = ['age', 'surgery_complexity']
# ordinal_columns = ['gender', 'race', 'marital_status', 'specialties']
# date_columns = ['scheduled_date']
# target_column = 'is_cancelled'

# dummy dataset
df = sns.load_dataset('titanic')
continuous_columns = ['age', 'fare', 'sibsp', 'parch']
ordinal_columns = ['sex', 'class']
date_columns = []
target_column = 'survived'

# optimization params
hyperparams_stepwise_groups = [
    ['learning_rate'],
    ['lambda_l1'], 
    ['lambda_l2'], 
    ['num_leaves'], 
    ['feature_fraction'], 
    ['bagging_fraction'], 
    ['bagging_freq'],
    ['min_child_samples'],
]
fixed_params={
        'verbosity': -1,
        'objective': 'binary',
        'is_unbalance': True,
        'n_estimators': 1000
    }
save_dir = '../runs/test_run'

In [9]:
best_params, best_value = get_lgbm_optimized_params(
    df,
    continuous_columns,
    ordinal_columns,
    date_columns,
    target_column,
    save_dir,
    hyperparams_stepwise_groups=hyperparams_stepwise_groups,
    # hyperparams_stepwise_groups=None,
    fixed_params=fixed_params,
    n_trials=10,
    random_state=0
    )

[I 2024-04-12 16:01:25,798] A new study created in memory with name: no-name-134358d8-9bee-4efb-bce9-e5e727ebdbe5


optimizing group: ['learning_rate']


[I 2024-04-12 16:01:26,288] Trial 0 finished with value: 0.859107618098913 and parameters: {'learning_rate': 0.1}. Best is trial 0 with value: 0.859107618098913.
[I 2024-04-12 16:01:26,869] Trial 1 finished with value: 0.8663487120198916 and parameters: {'learning_rate': 0.044303752452182654}. Best is trial 1 with value: 0.8663487120198916.
[I 2024-04-12 16:01:27,369] Trial 2 finished with value: 0.8572485809786075 and parameters: {'learning_rate': 0.13981961408994045}. Best is trial 1 with value: 0.8663487120198916.
[I 2024-04-12 16:01:27,892] Trial 3 finished with value: 0.8637218888715875 and parameters: {'learning_rate': 0.06431172050131989}. Best is trial 1 with value: 0.8663487120198916.
[I 2024-04-12 16:01:28,439] Trial 4 finished with value: 0.8646494005732256 and parameters: {'learning_rate': 0.0431171005868549}. Best is trial 1 with value: 0.8663487120198916.
[I 2024-04-12 16:01:28,933] Trial 5 finished with value: 0.8588957834285036 and parameters: {'learning_rate': 0.018662

best params: {'verbosity': -1, 'objective': 'binary', 'is_unbalance': True, 'n_estimators': 100, 'learning_rate': 0.044303752452182654}, best score: 0.8663487120198916
optimizing group: ['num_leaves']


[I 2024-04-12 16:01:31,554] Trial 0 finished with value: 0.8663487120198916 and parameters: {'num_leaves': 31}. Best is trial 0 with value: 0.8663487120198916.
[I 2024-04-12 16:01:32,098] Trial 1 finished with value: 0.8663487120198916 and parameters: {'num_leaves': 144}. Best is trial 0 with value: 0.8663487120198916.
[I 2024-04-12 16:01:32,770] Trial 2 finished with value: 0.8663487120198916 and parameters: {'num_leaves': 186}. Best is trial 0 with value: 0.8663487120198916.
[I 2024-04-12 16:01:33,333] Trial 3 finished with value: 0.8663487120198916 and parameters: {'num_leaves': 158}. Best is trial 0 with value: 0.8663487120198916.
[I 2024-04-12 16:01:33,889] Trial 4 finished with value: 0.8663487120198916 and parameters: {'num_leaves': 143}. Best is trial 0 with value: 0.8663487120198916.
[I 2024-04-12 16:01:34,470] Trial 5 finished with value: 0.8663487120198916 and parameters: {'num_leaves': 113}. Best is trial 0 with value: 0.8663487120198916.
[I 2024-04-12 16:01:35,058] Trial 6

...

best params: {'verbosity': -1, 'objective': 'binary', 'is_unbalance': True, 'n_estimators': 100, 'learning_rate': 0.044303752452182654, 'num_leaves': 31, 'feature_fraction': 0.7292881023563949, 'bagging_fraction': 1.0}, best score: 0.8665762442700545
optimizing group: ['bagging_freq']


[I 2024-04-12 16:01:46,578] Trial 0 finished with value: 0.8665762442700545 and parameters: {'bagging_freq': 0}. Best is trial 0 with value: 0.8665762442700545.
[I 2024-04-12 16:01:47,002] Trial 1 finished with value: 0.8665762442700545 and parameters: {'bagging_freq': 4}. Best is trial 0 with value: 0.8665762442700545.
[I 2024-04-12 16:01:47,830] Trial 2 finished with value: 0.8665762442700545 and parameters: {'bagging_freq': 6}. Best is trial 0 with value: 0.8665762442700545.
[I 2024-04-12 16:01:49,640] Trial 3 finished with value: 0.8665762442700545 and parameters: {'bagging_freq': 5}. Best is trial 0 with value: 0.8665762442700545.
[I 2024-04-12 16:01:50,086] Trial 4 finished with value: 0.8665762442700545 and parameters: {'bagging_freq': 4}. Best is trial 0 with value: 0.8665762442700545.
[I 2024-04-12 16:01:50,500] Trial 5 finished with value: 0.8665762442700545 and parameters: {'bagging_freq': 3}. Best is trial 0 with value: 0.8665762442700545.
[I 2024-04-12 16:01:50,944] Trial 

best params: {'verbosity': -1, 'objective': 'binary', 'is_unbalance': True, 'n_estimators': 100, 'learning_rate': 0.044303752452182654, 'num_leaves': 31, 'feature_fraction': 0.7292881023563949, 'bagging_fraction': 1.0, 'bagging_freq': 0}, best score: 0.8665762442700545
optimizing group: ['min_child_samples']


[I 2024-04-12 16:01:52,885] Trial 0 finished with value: 0.8665762442700545 and parameters: {'min_child_samples': 20}. Best is trial 0 with value: 0.8665762442700545.
[I 2024-04-12 16:01:53,204] Trial 1 finished with value: 0.860656989306946 and parameters: {'min_child_samples': 57}. Best is trial 0 with value: 0.8665762442700545.
[I 2024-04-12 16:01:53,508] Trial 2 finished with value: 0.8548377306465248 and parameters: {'min_child_samples': 73}. Best is trial 0 with value: 0.8665762442700545.
[I 2024-04-12 16:01:53,867] Trial 3 finished with value: 0.8599729865112089 and parameters: {'min_child_samples': 62}. Best is trial 0 with value: 0.8665762442700545.
[I 2024-04-12 16:01:54,343] Trial 4 finished with value: 0.860656989306946 and parameters: {'min_child_samples': 57}. Best is trial 0 with value: 0.8665762442700545.
[I 2024-04-12 16:01:54,942] Trial 5 finished with value: 0.8608343252084893 and parameters: {'min_child_samples': 45}. Best is trial 0 with value: 0.8665762442700545.


best params: {'verbosity': -1, 'objective': 'binary', 'is_unbalance': True, 'n_estimators': 100, 'learning_rate': 0.044303752452182654, 'num_leaves': 31, 'feature_fraction': 0.7292881023563949, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20}, best score: 0.8665762442700545


In [12]:
X, y = preprocess(df, continuous_columns, ordinal_columns, date_columns, target_column)
model = lgbm.LGBMClassifier(**best_params)
joblib.dump(model, '../runs/test_run/best_model.pkl')

['../runs/test_run/best_model.pkl']